### test data 22.1.1 ~ 22.4.27 긍부정 라벨링

In [3]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from konlpy.corpus import kolaw
import nltk

**구축한 단어사전 불러오기**

In [4]:
word_dict = pd.read_csv('./data/단어합본.csv')
word_dict = word_dict.drop('Unnamed: 0', axis=1)
word_dict

,word,polarity
0,ㅡㅡ,-1
1,ㅠㅠ,-1
2,ㅠ_ㅠ,-1
3,ㅠ,-1
4,ㅜㅡ,-1
...,...,...
18408,대출,0
18409,횡령,-2
18410,축하,1
18411,행복,2


In [5]:
# 반복문에 돌리기 위해서 df를 list로 바꾸는 작업
word_list = word_dict.values.tolist()
word_list

[['ㅡㅡ', -1],
 ['ㅠㅠ', -1],
 ['ㅠ_ㅠ', -1],
 ['ㅠ', -1],
 ['ㅜㅡ', -1],
 ['ㅜㅜ', -1],
 ['ㅜ_ㅜ', -1],
 ['ㅜ.ㅜ', -1],
 ['ㅜ', -1],
 ['ㅗ', -1],
 ['힘찬 기운이', 2],
 ['힘찬', 2],
 ['힘차다', 2],
 ['힘차고 튼튼하다', 2],
 ['힘차고 대단한', 2],
 ['힘차고', 2],
 ['힘차게 진행되는', 2],
 ['힘차게 일어남', 1],
 ['힘차게', 2],
 ['힘이나 권세 위력 따위가 없이', -1],
 ['힘이 있다', 1],
 ['힘이 있고', 1],
 ['힘이 있게', 1],
 ['힘이 없음을', -1],
 ['힘이 없어서', -1],
 ['힘이 없어', -1],
 ['힘이 없다', -1],
 ['힘이 없는 사람을', -1],
 ['힘이 없는', -1],
 ['힘이 없거나', -1],
 ['힘이 약해진', -1],
 ['힘이 빠져', -1],
 ['힘이 뛰어나다', 1],
 ['힘이 들거나', -1],
 ['힘이 되어', 1],
 ['힘이 다 빠져', -1],
 ['힘이 기운차게', 2],
 ['힘이 강하다', 2],
 ['힘이 가득하고', 1],
 ['힘을 합하여', 1],
 ['힘을 잃고', -1],
 ['힘을 잃게', -1],
 ['힘을 잃거나', -1],
 ['힘을 못 쓰게', -1],
 ['힘을 들이고', -1],
 ['힘을 들였으나', -1],
 ['힘을 다함', 1],
 ['힘을 다하여', 1],
 ['힘을 다하다', 1],
 ['힘을 가지고', 1],
 ['힘에 부치는 데가 있다', -2],
 ['힘에 부쳐', -2],
 ['힘에 벅차', -2],
 ['힘에 눌려', -2],
 ['힘에 겹게', -2],
 ['힘에 겹거나', -2],
 ['힘에 겨워', -2],
 ['힘없이 흐느적흐느적', -1],
 ['힘없이 행동하다', -1],
 ['힘없이 행동하는', -1],
 ['힘없이 처질', -1],
 ['힘없이 주저앉는', -

### 올해 뉴스 데이터 합치기

In [12]:
# 4.13까지 데이터 추출
front_data = pd.read_csv('./data/all_news_data.csv', encoding= 'cp949')
front_data = front_data.drop('Unnamed: 0', axis=1)
front_data = front_data.drop('...1', axis=1)
front_data = front_data[13216:14890]
front_data

,제목,날짜
13216,‘신규확진 7천명’인데…오미크론 대응 준비 안 된 ‘호흡기 클리닉’,2022.01.02.
13217,"한화손보, 모바일 가입 간소화…디지털 전용 운전자보험 판매",2022.01.02.
13218,제2의 아이유 꿈꾸나…윤종신이 찍은 LG 가상인간 싱어송라이터 '래아',2022.01.02.
13219,"해진공, 한국형 컨테이너 운임지수 시범 도입···SCFI 넘을까",2022.01.02.
13220,류영준 카카오 대표 내정자 사퇴,2022.01.03.
...,...,...
14885,인간중심 기술 위한 한국 기업의 역할[기고/필립 코틀러],2022.04.11.
14886,신용 하위50% 늘어난다...인터넷銀 중·저신용 대출확대 '청신호',2022.04.11.
14887,"""바이오가 되겠냐 했는데 이젠 금맥""…롯데·GS·신세계도 참전",2022.04.11.
14888,"""대출 받아가세요""…굳게 닫혔던 은행 문이 다시 열렸다",2022.04.11.


In [13]:
back_data = pd.read_csv('./data/april.csv')
back_data = back_data.drop('Unnamed: 0', axis=1)
back_data

,제목,날짜
0,"""디지털 기술로 기업간 합종연횡 가속화"" [제23회 서울국제금융포럼]",2022.04.14.
1,오늘의 증시 일정 (4월 19일),2022.04.14.
2,"5부능선 앞둔 망 이용대가법, 논의 향방은",2022.04.14.
3,"타다, 대형택시 호출 서비스 ‘타다 넥스트’로 재시동",2022.04.14.
4,삼성·LG·네이버 이어 카카오도…AI 스타트업 '노타'에 꽂혔다,2022.04.14.
...,...,...
245,'망 무임승차 방지법' 진통 끝 상정... 국회 문턱 넘을까,2022.04.27.
246,"[단독] 괌에서 원격근무도 된다…라인, 재택공간 제한 풀었다",2022.04.27.
247,靑 내달 10일 정오부터 무료 개방…27일 사전 신청,2022.04.27.
248,‘고어방 파문’ 1년 만에 다시 등장한 ‘고양이 학대방’,2022.04.27.


In [14]:
### 위의 두 data 합치기
news_data =  pd.concat([front_data, back_data], ignore_index=True)
news_data

,제목,날짜
0,‘신규확진 7천명’인데…오미크론 대응 준비 안 된 ‘호흡기 클리닉’,2022.01.02.
1,"한화손보, 모바일 가입 간소화…디지털 전용 운전자보험 판매",2022.01.02.
2,제2의 아이유 꿈꾸나…윤종신이 찍은 LG 가상인간 싱어송라이터 '래아',2022.01.02.
3,"해진공, 한국형 컨테이너 운임지수 시범 도입···SCFI 넘을까",2022.01.02.
4,류영준 카카오 대표 내정자 사퇴,2022.01.03.
...,...,...
1919,'망 무임승차 방지법' 진통 끝 상정... 국회 문턱 넘을까,2022.04.27.
1920,"[단독] 괌에서 원격근무도 된다…라인, 재택공간 제한 풀었다",2022.04.27.
1921,靑 내달 10일 정오부터 무료 개방…27일 사전 신청,2022.04.27.
1922,‘고어방 파문’ 1년 만에 다시 등장한 ‘고양이 학대방’,2022.04.27.


### 합친 뉴스 파일을 토큰화 하는 작업

In [15]:
okt = Okt()

In [16]:
title = news_data['제목']

In [17]:
# 불용어 제거

for i in range(len(title)):
    title[i] = re.sub(r'[!"#$%&\'()*+,-./:·;<=>?@\[\]^_\`{|}~\\\\]','', title[i])
    
news_data['제목'] = title
news_data

,제목,날짜
0,‘신규확진 7천명’인데…오미크론 대응 준비 안 된 ‘호흡기 클리닉’,2022.01.02.
1,한화손보 모바일 가입 간소화…디지털 전용 운전자보험 판매,2022.01.02.
2,제2의 아이유 꿈꾸나…윤종신이 찍은 LG 가상인간 싱어송라이터 래아,2022.01.02.
3,해진공 한국형 컨테이너 운임지수 시범 도입SCFI 넘을까,2022.01.02.
4,류영준 카카오 대표 내정자 사퇴,2022.01.03.
...,...,...
1919,망 무임승차 방지법 진통 끝 상정 국회 문턱 넘을까,2022.04.27.
1920,단독 괌에서 원격근무도 된다…라인 재택공간 제한 풀었다,2022.04.27.
1921,靑 내달 10일 정오부터 무료 개방…27일 사전 신청,2022.04.27.
1922,‘고어방 파문’ 1년 만에 다시 등장한 ‘고양이 학대방’,2022.04.27.


In [18]:
okt_list = []

for i in range(len(title)):
    okt_list.append(okt.morphs(title[i]))
    
print(okt_list)

[['‘', '신규', '확진', '7천', '명', '’', '인데', '…', '오미크론', '대응', '준비', '안', '된', '‘', '호흡기', '클리', '닉', '’'], ['한화', '손보', '모바일', '가입', '간소화', '…', '디지털', '전용', '운전자', '보험', '판매'], ['제', '2', '의', '아이유', '꿈꾸나', '…', '윤종신', '이', '찍은', 'LG', '가상', '인간', '싱어송라이터', '래', '아'], ['해', '진공', '한국', '형', '컨테이너', '운임', '지수', '시범', '도입', 'SCFI', '넘을까'], ['류영준', '카카오', '대표', '내정자', '사퇴'], ['카카오', '뱅크', '페이', '부진', '의', '늪', '빠졌다', '목표', '주', '가도', '하향'], ['리더십', '위기', '빠진', '류영준', '카카오', '노조', '대표', '선임', '철회', '하', '라'], ['“', '크래', '프톤', '카카오', '줍자', '”', '연초', '‘', '빚투', '’', '다시', '늘었다'], ['본인', '확인', '기관', '심사', '기준', '완화', '됐지만', '…', '네이버', '카카오', '부적합', '항목', '해소', '해야'], ['이승효', '카카오', '페이', '증권', '공동', '대표', '내정자', '카카오', '페이', '주식', '거래', '출시', '…', '주린이도', '편하게', '사용', '가능'], ['여전히', '방역', '패스', '안되는', '‘', '클린', '강원', '패스포트'], ['월', '매출', '6만원', '→', '700만원', '됐어요', '족집게', '강의', '하나', '듣고', '대박났다'], ['DSR', '규제', '신용', '대출', '한도', '제한', '…', '대출', '문턱', '더', '높아진다'], ['미성년', '계좌', '2년', '간'

In [19]:
news_data['token'] = okt_list
news_data = news_data[['날짜','제목','token']]
news_data

,날짜,제목,token
0,2022.01.02.,‘신규확진 7천명’인데…오미크론 대응 준비 안 된 ‘호흡기 클리닉’,"[‘, 신규, 확진, 7천, 명, ’, 인데, …, 오미크론, 대응, 준비, 안, ..."
1,2022.01.02.,한화손보 모바일 가입 간소화…디지털 전용 운전자보험 판매,"[한화, 손보, 모바일, 가입, 간소화, …, 디지털, 전용, 운전자, 보험, 판매]"
2,2022.01.02.,제2의 아이유 꿈꾸나…윤종신이 찍은 LG 가상인간 싱어송라이터 래아,"[제, 2, 의, 아이유, 꿈꾸나, …, 윤종신, 이, 찍은, LG, 가상, 인간,..."
3,2022.01.02.,해진공 한국형 컨테이너 운임지수 시범 도입SCFI 넘을까,"[해, 진공, 한국, 형, 컨테이너, 운임, 지수, 시범, 도입, SCFI, 넘을까]"
4,2022.01.03.,류영준 카카오 대표 내정자 사퇴,"[류영준, 카카오, 대표, 내정자, 사퇴]"
...,...,...,...
1919,2022.04.27.,망 무임승차 방지법 진통 끝 상정 국회 문턱 넘을까,"[망, 무임, 승차, 방지법, 진통, 끝, 상정, 국회, 문턱, 넘을까]"
1920,2022.04.27.,단독 괌에서 원격근무도 된다…라인 재택공간 제한 풀었다,"[단독, 괌, 에서, 원, 격, 근무, 도, 된다, …, 라인, 재택, 공간, 제한..."
1921,2022.04.27.,靑 내달 10일 정오부터 무료 개방…27일 사전 신청,"[靑, 내달, 10일, 정오, 부터, 무료, 개방, …, 27일, 사전, 신청]"
1922,2022.04.27.,‘고어방 파문’ 1년 만에 다시 등장한 ‘고양이 학대방’,"[‘, 고어, 방, 파문, ’, 1년, 만에, 다시, 등장, 한, ‘, 고양이, 학..."


### 뉴스 토큰과 사전파일 대조하여 가중치 더하기

In [20]:
count = 0
a = []

for i in news_data['token']:
    li = []
    for j in i:
        for k in word_list:
            if j == k[0]:
                count += k[1]
                
    li.append(count)
    a.append(li)
    count = 0

a

[[-1],
 [1],
 [0],
 [0],
 [-2],
 [-3],
 [-6],
 [0],
 [1],
 [4],
 [0],
 [1],
 [-6],
 [1],
 [-2],
 [1],
 [-1],
 [1],
 [-2],
 [-4],
 [-1],
 [-3],
 [-3],
 [-2],
 [0],
 [0],
 [0],
 [6],
 [-1],
 [-1],
 [0],
 [-2],
 [0],
 [0],
 [2],
 [-1],
 [0],
 [-4],
 [1],
 [1],
 [1],
 [1],
 [-1],
 [5],
 [0],
 [4],
 [2],
 [4],
 [3],
 [-1],
 [0],
 [2],
 [0],
 [2],
 [-6],
 [1],
 [-10],
 [-1],
 [-4],
 [3],
 [0],
 [-2],
 [0],
 [3],
 [1],
 [-5],
 [0],
 [0],
 [-3],
 [-3],
 [2],
 [2],
 [0],
 [-5],
 [2],
 [-2],
 [-3],
 [2],
 [-1],
 [-1],
 [0],
 [-2],
 [-1],
 [0],
 [1],
 [0],
 [6],
 [1],
 [-5],
 [0],
 [2],
 [-2],
 [-1],
 [0],
 [-4],
 [2],
 [1],
 [0],
 [3],
 [-1],
 [-2],
 [2],
 [-1],
 [1],
 [0],
 [0],
 [4],
 [-4],
 [0],
 [-1],
 [-1],
 [0],
 [1],
 [1],
 [1],
 [-1],
 [-5],
 [2],
 [0],
 [-5],
 [2],
 [1],
 [1],
 [3],
 [-2],
 [-1],
 [1],
 [0],
 [3],
 [9],
 [2],
 [0],
 [1],
 [-1],
 [4],
 [-2],
 [2],
 [0],
 [-4],
 [0],
 [-5],
 [0],
 [1],
 [-1],
 [1],
 [-8],
 [-4],
 [-3],
 [3],
 [-1],
 [1],
 [2],
 [0],
 [0],
 [0],
 [-4],
 [1

In [21]:
# 길이 같음 -> 맞게 뽑힌 걸 알 수 있음
print(len(a))
print(len(news_data['token']))

1924
1924


In [22]:
## 긍정/부정/중립 분류하기 편하게 int 형식으로 바꿈

a_li = []

for i in a:    
    i = "".join([str(_) for _ in i])
    i = int(i)
    a_li.append(i)

len(a_li)

1924

In [23]:
news_data['가중치'] = a_li
news_data = news_data[['날짜','제목','token','가중치']]
news_data

C:\Users\imj\AppData\Local\Temp\ipykernel_18020\12472749.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['가중치'] = a_li


,날짜,제목,token,가중치
0,2022.01.02.,‘신규확진 7천명’인데…오미크론 대응 준비 안 된 ‘호흡기 클리닉’,"[‘, 신규, 확진, 7천, 명, ’, 인데, …, 오미크론, 대응, 준비, 안, ...",-1
1,2022.01.02.,한화손보 모바일 가입 간소화…디지털 전용 운전자보험 판매,"[한화, 손보, 모바일, 가입, 간소화, …, 디지털, 전용, 운전자, 보험, 판매]",1
2,2022.01.02.,제2의 아이유 꿈꾸나…윤종신이 찍은 LG 가상인간 싱어송라이터 래아,"[제, 2, 의, 아이유, 꿈꾸나, …, 윤종신, 이, 찍은, LG, 가상, 인간,...",0
3,2022.01.02.,해진공 한국형 컨테이너 운임지수 시범 도입SCFI 넘을까,"[해, 진공, 한국, 형, 컨테이너, 운임, 지수, 시범, 도입, SCFI, 넘을까]",0
4,2022.01.03.,류영준 카카오 대표 내정자 사퇴,"[류영준, 카카오, 대표, 내정자, 사퇴]",-2
...,...,...,...,...
1919,2022.04.27.,망 무임승차 방지법 진통 끝 상정 국회 문턱 넘을까,"[망, 무임, 승차, 방지법, 진통, 끝, 상정, 국회, 문턱, 넘을까]",-2
1920,2022.04.27.,단독 괌에서 원격근무도 된다…라인 재택공간 제한 풀었다,"[단독, 괌, 에서, 원, 격, 근무, 도, 된다, …, 라인, 재택, 공간, 제한...",0
1921,2022.04.27.,靑 내달 10일 정오부터 무료 개방…27일 사전 신청,"[靑, 내달, 10일, 정오, 부터, 무료, 개방, …, 27일, 사전, 신청]",0
1922,2022.04.27.,‘고어방 파문’ 1년 만에 다시 등장한 ‘고양이 학대방’,"[‘, 고어, 방, 파문, ’, 1년, 만에, 다시, 등장, 한, ‘, 고양이, 학...",-1


In [27]:
# 점수가 -1이하면 부정, 0이면 중립, 1이상이면 긍정으로 분류

sort_li = []

for i in news_data['가중치']:
    if i >= 1:
        sort_li.append('1')
    elif i == 0 :
        sort_li.append('0')
    else:
        sort_li.append('-1')

len(sort_li)

1924

In [42]:
news_data['분류'] = sort_li

In [47]:
# news_data = news_data[['날짜','제목','token','가중치','분류']]
# news_data = news_data.drop('token', axis=1)
# news_data = news_data.drop('분류', axis=1)
news_data.columns = ['date', 'text', 'score', 'senti']
news_data

,date,text,score,senti
0,2022.01.02.,‘신규확진 7천명’인데…오미크론 대응 준비 안 된 ‘호흡기 클리닉’,-1,-1
1,2022.01.02.,한화손보 모바일 가입 간소화…디지털 전용 운전자보험 판매,1,1
2,2022.01.02.,제2의 아이유 꿈꾸나…윤종신이 찍은 LG 가상인간 싱어송라이터 래아,0,0
3,2022.01.02.,해진공 한국형 컨테이너 운임지수 시범 도입SCFI 넘을까,0,0
4,2022.01.03.,류영준 카카오 대표 내정자 사퇴,-2,-1
...,...,...,...,...
1919,2022.04.27.,망 무임승차 방지법 진통 끝 상정 국회 문턱 넘을까,-2,-1
1920,2022.04.27.,단독 괌에서 원격근무도 된다…라인 재택공간 제한 풀었다,0,0
1921,2022.04.27.,靑 내달 10일 정오부터 무료 개방…27일 사전 신청,0,0
1922,2022.04.27.,‘고어방 파문’ 1년 만에 다시 등장한 ‘고양이 학대방’,-1,-1


In [46]:
# 최종 저장
news_data.to_csv('./data/긍부정분류_test.csv')